This notebook is only used for Data Exploration Purposes.


Assumptions:
- Only stations in Flanders and Brussels are taken into account. Making abstraction of the fact that measurements in stations in Wallonia could also have an impact on air quality in Flanders, particularly in regions close to the Walloon border. (https://data.europa.eu/data/datasets/https-www-odwb-be-explore-dataset-regionsgeweste-belgium-?locale=en)
- Mapping of pollutant parameters based on "https://geo.irceline.be/sos/api/v1/phenomena"
- Mapped certain labels to cities that are more logical
- Did not filter out pollution values of 0 or outliers as their meaning depends on the Pollutor and I don't have the required knowledge to treath them
- ...

# Setup and configuration

In [1]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import plotly.express as px
import plotly.graph_objects as go
import datetime

from typing import List, Dict

# Read data

In [2]:
base_url = "https://geo.irceline.be/sos/api/v1"
stations_url = f"{base_url}/stations"
time_series_url = f"{base_url}/timeseries"

In [3]:
def fetch_url(url: str) -> List[Dict]:
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

## Stations data

In [5]:
def get_flanders_border(include_brussels: bool = True) -> gpd.GeoDataFrame:
    regions_to_include = ["vlaams gewest"]
    if include_brussels:
        regions_to_include.append("brussels hoofdstedelijk gewest")
    
    regions_gdf = gpd.read_file("regionsgeweste-belgium.geojson")[["geometry", "reg_name_lower_nl"]]
    return regions_gdf[regions_gdf["reg_name_lower_nl"].isin(regions_to_include)]

In [6]:
def is_flanders(geom: List[int], city: str) -> bool:
    if not geom or len(geom) < 2 or geom[0] is None or geom[1] is None:
        return False

    flanders_regions_gpd = get_flanders_border()
    return flanders_regions_gpd.contains(Point(geom[0], geom[1])).any()


def extract_city(label: str) -> str:
    city_mapping_dict = {
        "48R236 - Aeroport 2": "Liège",
        "48R516 - Aeroport 2": "Charleroi",
        "48R237 - Aeroport 1": "Liège",
        "48R515 - Aeroport 1": "Charleroi",
        "Arts-Loi": "Brussel",
        "Bruxelles": "Brussel",
        "Maerlant": "Antwerpen",
    }
    
    try:
        name_part = label.split(" - ", 1)[1]
        city = name_part.split("(", 1)[0].strip().capitalize()

        for city_mapping in list(city_mapping_dict.keys()):
            if city_mapping in label:
                city = city_mapping_dict.get(city_mapping)
        return city

    except IndexError:
        return None

    
def get_and_parse_station_data(stations_url: str) -> pd.DataFrame:
    stations_raw = fetch_url(stations_url)

    records = []
    for station in stations_raw:
        props = station.get("properties")
        geom = station.get("geometry").get("coordinates")
        
        label = props.get("label")
        city = extract_city(label)

        if not is_flanders(geom, city):
            continue

        records.append({
            "station_id": props.get("id"),
            "station_label": label,
            "station_longitude": geom[0],
            "station_latitude": geom[1],
            "station_city": city,
        })

    return pd.DataFrame(records)

In [7]:
get_and_parse_station_data(stations_url)

Skipping field reg_code: unsupported OGR type: 5
Skipping field reg_name_fr: unsupported OGR type: 5
Skipping field reg_name_nl: unsupported OGR type: 5
Skipping field reg_name_de: unsupported OGR type: 5
Skipping field reg_code: unsupported OGR type: 5
Skipping field reg_name_fr: unsupported OGR type: 5
Skipping field reg_name_nl: unsupported OGR type: 5
Skipping field reg_name_de: unsupported OGR type: 5
Skipping field reg_code: unsupported OGR type: 5
Skipping field reg_name_fr: unsupported OGR type: 5
Skipping field reg_name_nl: unsupported OGR type: 5
Skipping field reg_name_de: unsupported OGR type: 5
Skipping field reg_code: unsupported OGR type: 5
Skipping field reg_name_fr: unsupported OGR type: 5
Skipping field reg_name_nl: unsupported OGR type: 5
Skipping field reg_name_de: unsupported OGR type: 5
Skipping field reg_code: unsupported OGR type: 5
Skipping field reg_name_fr: unsupported OGR type: 5
Skipping field reg_name_nl: unsupported OGR type: 5
Skipping field reg_name_de:

,station_id,station_label,station_longitude,station_latitude,station_city
0,1031,40AL02 - Beveren,4.234833,51.304521,Beveren
1,1032,40AL03 - Beveren,4.201460,51.253965,Beveren
2,1033,40AL04 - Beveren,4.293329,51.290668,Beveren
3,1034,40AL05 - Beveren,4.278890,51.263118,Beveren
4,1774,40AT83 - BERENDRECHT,4.304989,51.339803,Berendrecht
...,...,...,...,...,...
94,1760,FLM3M3 - Antwerpen (3M - Fence Line ZO),4.340936,51.229160,Antwerpen
95,1761,FLM3M4 - Antwerpen (3M - Fence Line Z),4.335160,51.229270,Antwerpen
96,1762,FLM3M5 - Antwerpen (3M - Fence Line ZW),4.329620,51.230220,Antwerpen
97,1763,FLM3M6 - Antwerpen (3M - Fence Line W),4.326684,51.232600,Antwerpen


## Time series data

In [ ]:
def extract_pollutant_label(label: str) -> bool:
    target_pollutants_dict = {
        "pm10": "pm10",
        "10 µm": "pm10",
        "2.5 µm": "ppm25",
        "nitrogen dioxide": "no2",
        "carbon dioxide": "co2",
        "sulphur dioxide": "so2",
    }

    # Can also be based on this mapping https://geo.irceline.be/sos/api/v1/phenomena
    label_lower = label.lower()
    for p in list(target_pollutants_dict.keys()):
        if p in label_lower:
            return target_pollutants_dict.get(p)
    return None


def timestamp_to_str(timestamp: str) -> str:
    return datetime.datetime.fromtimestamp(int(timestamp) / 1000, datetime.UTC).strftime('%Y-%m-%d %H:%M:%S')


def fetch_timeseries_details(timeseries_id: int, time_series_url: str) -> pd.DataFrame:
    ts_details_url = f"{time_series_url}/{timeseries_id}"

    try:
        timeseries_details_raw = fetch_url(ts_details_url)
        first = timeseries_details_raw.get("firstValue", {})
        last = timeseries_details_raw.get("lastValue", {})
        return {
            "timeseries_id": timeseries_id,
            "first_value": first.get("value"),
            "first_timestamp": timestamp_to_str(first.get("timestamp")),
            "last_value": last.get("value"),
            "last_timestamp": timestamp_to_str(last.get("timestamp"))
        }

    except Exception as e:
        print(f"Error fetching details for ID {timeseries_id}: {e}")
        return {
            "timeseries_id": timeseries_id,
            "first_value": None,
            "first_timestamp": None,
            "last_value": None,
            "last_timestamp": None
        }

    
def add_timeseries_details(timeseries_pd: pd.DataFrame, time_series_url: str) -> pd.DataFrame:
    ts_details = []
    for ts_id in timeseries_pd["timeseries_id"]:
        ts_details.append(fetch_timeseries_details(ts_id, time_series_url))
    
    ts_details_pd = pd.DataFrame(ts_details)
    return pd.merge(timeseries_pd, ts_details_pd, on="timeseries_id", how="left")

        
def get_and_parse_timeseries_data(time_series_url: str) -> pd.DataFrame:
    timeseries_raw = fetch_url(time_series_url)
    
    records = []
    for ts in timeseries_raw:
        station_id = ts.get("station", {}).get("properties", {}).get("id")
        pollutant_parameter = extract_pollutant_label(ts.get("label", ""))

        if pollutant_parameter is None:
            continue

        records.append({
            "timeseries_id": ts.get("id"),
            "pollution_label": ts.get("label"),
            "pollutant_parameter": pollutant_parameter,
            "uom": ts.get("uom"),
            "station_id": station_id,
        })
    timeseries_pd = pd.DataFrame(records)
    return add_timeseries_details(timeseries_pd, time_series_url)

# Adding the details might not be required actually. Perhaps only for the 1st question to answer as most have recent datapoints

In [10]:
get_and_parse_timeseries_data(time_series_url)

,timeseries_id,pollution_label,pollutant_parameter,uom,station_id,first_value,first_timestamp,last_value,last_timestamp
0,6612,Carbon Dioxide 6612 - This model 41H - procedu...,co2,ppm,1119,416.0,2012-05-29 01:00:00,423.0,2021-06-16 10:00:00
1,6619,Carbon Dioxide 6619 - This model 41H - procedu...,co2,ppm,1122,417.0,2012-05-29 01:00:00,417.0,2022-01-20 10:00:00
2,100019,"Nitrogen dioxide 100019 - - procedure, 42R842...",no2,µg/m³,1744,18.5,2020-08-04 01:00:00,13.5,2025-07-29 14:00:00
3,100026,Nitrogen dioxide 100026 - THIS 42C - procedure...,no2,µg/m³,1740,12.5,2021-06-11 01:00:00,21.5,2024-10-29 08:00:00
4,100032,Nitrogen dioxide 100032 - vanaf 970317 AC-31M ...,no2,µg/m³,1741,26.0,2021-09-18 01:00:00,34.5,2025-07-29 14:00:00
...,...,...,...,...,...,...,...,...,...
299,7120,Sulphur dioxide 7120 - ENVIRONNEMENT AF21M - p...,so2,µg/m³,1214,1.0,2012-05-31 01:00:00,22.0,2020-01-17 08:00:00
300,7130,"Sulphur dioxide 7130 - MELOY SA - procedure, 4...",so2,µg/m³,1217,0.0,2012-05-31 01:00:00,4.0,2013-03-04 13:00:00
301,7147,"Sulphur dioxide 7147 - MELOY SA - procedure, 4...",so2,µg/m³,1219,0.0,2012-05-31 01:00:00,0.0,2025-07-29 14:00:00
302,7158,"Sulphur dioxide 7158 - MELOY SA - procedure, 4...",so2,µg/m³,1221,3.0,2012-05-31 01:00:00,0.0,2025-07-29 14:00:00


## Hourly data

In [ ]:
def get_and_parse_timeseries_hourly_data(ts_id: int, start_date: str, end_date: str) -> pd.DataFrame:
    time_series_hourly_url = f"https://geo.irceline.be/sos/api/v1/timeseries/{ts_id}/getData?timespan={start_date}TZ/{end_date}TZ"
    timeseries_hourly_raw = fetch_url(time_series_hourly_url)
    
    hourly_ts_pd = pd.DataFrame(timeseries_hourly_raw['values'])
    hourly_ts_pd['timestamp'] = pd.to_datetime(hourly_ts_pd['timestamp'], unit='ms')
    hourly_ts_pd['timeseries_id'] = ts_id

    return hourly_ts_pd.rename(columns={'timestamp': 'datetime', 'value': 'pollution_value'})[["timeseries_id", "datetime", "pollution_value"]]


In [29]:
start_date = "2021-06-01"
end_date = "2021-06-16"
timeseries_id = "6612"

get_and_parse_timeseries_hourly_data(timeseries_id, start_date, end_date).head()

,timeseries_id,datetime,pollution_value
0,6612,2021-06-01 00:00:00,433.5
1,6612,2021-06-01 01:00:00,438.0
2,6612,2021-06-01 02:00:00,444.0
3,6612,2021-06-01 03:00:00,450.0
4,6612,2021-06-01 04:00:00,456.0


# Analyse the data

## Stations data
- ID / Label all unique?? --> Yes
- Geometry always the same format? z-coordinate always 'NaN'?? --> Yes
- Locations all in Flanders??  --> In the whole of Belgium
- Type of geometry always 'Point'? --> Yes
- Type of entry always 'Feature'? --> Yes

In [30]:
stations_pd = get_and_parse_station_data(stations_url)
stations_pd

Skipping field reg_code: unsupported OGR type: 5
Skipping field reg_name_fr: unsupported OGR type: 5
Skipping field reg_name_nl: unsupported OGR type: 5
Skipping field reg_name_de: unsupported OGR type: 5
Skipping field reg_code: unsupported OGR type: 5
Skipping field reg_name_fr: unsupported OGR type: 5
Skipping field reg_name_nl: unsupported OGR type: 5
Skipping field reg_name_de: unsupported OGR type: 5
Skipping field reg_code: unsupported OGR type: 5
Skipping field reg_name_fr: unsupported OGR type: 5
Skipping field reg_name_nl: unsupported OGR type: 5
Skipping field reg_name_de: unsupported OGR type: 5
Skipping field reg_code: unsupported OGR type: 5
Skipping field reg_name_fr: unsupported OGR type: 5
Skipping field reg_name_nl: unsupported OGR type: 5
Skipping field reg_name_de: unsupported OGR type: 5
Skipping field reg_code: unsupported OGR type: 5
Skipping field reg_name_fr: unsupported OGR type: 5
Skipping field reg_name_nl: unsupported OGR type: 5
Skipping field reg_name_de:

,station_id,station_label,station_longitude,station_latitude,station_city
0,1031,40AL02 - Beveren,4.234833,51.304521,Beveren
1,1032,40AL03 - Beveren,4.201460,51.253965,Beveren
2,1033,40AL04 - Beveren,4.293329,51.290668,Beveren
3,1034,40AL05 - Beveren,4.278890,51.263118,Beveren
4,1774,40AT83 - BERENDRECHT,4.304989,51.339803,Berendrecht
...,...,...,...,...,...
94,1760,FLM3M3 - Antwerpen (3M - Fence Line ZO),4.340936,51.229160,Antwerpen
95,1761,FLM3M4 - Antwerpen (3M - Fence Line Z),4.335160,51.229270,Antwerpen
96,1762,FLM3M5 - Antwerpen (3M - Fence Line ZW),4.329620,51.230220,Antwerpen
97,1763,FLM3M6 - Antwerpen (3M - Fence Line W),4.326684,51.232600,Antwerpen


In [31]:
fig = px.scatter_map(
    stations_pd,
    lat=stations_pd.station_latitude,
    lon=stations_pd.station_longitude,
    hover_name=stations_pd.station_label,
    zoom=6,
    height=600,
    width=800,
)

fig.update_layout(title="Air Quality Monitoring Stations in Flanders & Brussels")
fig.show()

## Time series data
- ID / Label all unique?? --> Yes
- 'station'values align with station dataset? Are there stations without sime series values or time series stations not in stations dataset? --> They all allign
- Period of the data? --> Available per hour

In [45]:
date_to_check = datetime.date.today().strftime("%Y-%m-%d")
pollution_parameter_to_check = "ppm25"

parsed_timeseries_latest_data = (
    get_and_parse_timeseries_data(time_series_url)
    .query(f"last_timestamp.str.contains('{date_to_check}')")
    .query(f"pollutant_parameter == '{pollution_parameter_to_check}'")
)[["station_id", "last_value"]]

In [46]:
# Check the most polluting cities

pollution_per_city = (
    pd.merge(stations_pd, parsed_timeseries_latest_data, on="station_id")
    .groupby("station_city")
    .agg(
        pollution=("last_value", "max"),
        nb_stations=("station_id", "count")
    )
    .sort_values(by="pollution", ascending=False)
    .reset_index()
)

pollution_per_city

,station_city,pollution,nb_stations
0,Menen,13.384,1
1,Hasselt,12.953,1
2,Roeselare,12.953,2
3,Zwevegem,12.089,1
4,Mechelen,11.657,1
5,Vilvoorde,11.657,1
6,Aarschot,11.657,1
7,Boom,11.226,1
8,Houthalen-helchteren,10.794,1
9,Oostrozebeke,10.362,1
